#  Data Analyst Portfolio Project - Automatic Notification Email SLA Summary Report

### the purpose is to send daily report with summarize of number of applications that meet the SLA and not of each stage of the existing application in BMI where the source data is from datawarehouse

### Table datawarehouse
<img src='inq_fos.png'
    style="width:450px; float: left; margin: 0 20px 20px 20px;"></img>
<img src='track_fos.png'
    style="width:450px; float: left; margin: 0 20px 20px 20px;"></img>

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

### Task Scheduler and Output
<img src='task_sch.png'
    style="width:450px; float: left; margin: 0 20px 20px 20px;"></img>
<img src='rep_apl.png'
    style="width:450px; float: left; margin: 0 20px 20px 20px;"></img>
    

## Business Request & User Stories

The business request for this data analyst project was an executive report for stages managers. Based on the request that was made from the business we following user stories were defined to fulfill delivery and ensure that acceptance criteria’s were maintained throughout the project.


 | # | As a (role)  | I want (request/demand) | So that i (user value) | Acceptance Criteria   |
|-:|:-:|:-|:-|:------|
| 1 | Head Of Department  | To get a overview of application process | To check which stages are over SLA | Send automatic Email report which updates data once a day |
| 2 | Manager  | To get a overview which stages need to be improved | To check which stages are over SLA   | Send automatic Email report which updates data once a day |
| 3 | PIC  | To get a overview which stages need to be improved | To improve the over SLA stages | Send automatic Email report which updates data once a day |

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## Hands on! 

### here are the code in python 

In [ ]:
#Jype1
import pandas as pd
import xlsxwriter
import jaydebeapi, os
from datetime import datetime, timedelta
import numpy as np
import datetime as dt
import xlwings as xw
import shutil
import openpyxl
import os
import win32com.client as client

from PIL import ImageGrab
pd. set_option('display.max_columns', 500)

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Connect and Loading the data from DWH:

In [ ]:
#Connect Python with DWH Netteza

dsn_database = "DWHDBPRD"
dsn_hostname = "10.55.53.xx"
dsn_port = "54xx"
dsn_uid = "Bxx_FOP"
dsn_pwd = "wingmanxxx"
jdbc_driver_name = "org.netezza.Driver"
jdbc_driver_loc = os.path.join('C:\\Intel\\driver_dbeaver\\nzjdbc.jar')

connection_string='jdbc:netezza://'+dsn_hostname+':'+dsn_port+'/'+dsn_database

url = '{0}:user={1};password={2}'.format(connection_string, dsn_uid, dsn_pwd)
print("Connection String: " + connection_string)

conn = jaydebeapi.connect(jdbc_driver_name, connection_string, {'user': dsn_uid, 'password': dsn_pwd},
jars=jdbc_driver_loc)

In [ ]:
#Collect data from table in DWH using SQL Query

#Laporan Tracking FOS
curs = conn.cursor()
curs.execute("SELECT * FROM xxxRPT.LAPORAN_TRACKING_xxx;")
result = curs.fetchall()
laporan_tracking_fos = pd.DataFrame(result)

# adding column name to the respective columns
laporan_tracking_fos.columns =['CREATE_TIME','AREAID','CABANG','NIK_MARKETING','APPLICATIONID','KTP','NAMA_NASABAH','SEGMENT','PRODUK','PROGRAM','TANGGAL_INPUT',
               'RM_1','RM_2','BM_SCREENING','PRESCREEN','ENTRI_DATA','FRAUD_SCREENING','FRAUD_CHECKING','FRAUD_ANALYST','WAITING_FOR_VERIFICATION',
               'VERIFICATION_REVIEW','ANALISA_KREDIT','FA_MANAGER','KOMITE_BISNIS','KOMITE_RISK','OL_PREPARATION','LEGAL','REVIEW_PENGIKATAN',
               'ADMIN_DROPPING','SETTLEMENT','SETTLEMENT_SPV','CANCEL','REJECT','BOOKED','LATEST_POSITION','CURRENT_USER_ID','CURRENT_USER1',
               'TANGGAL_POSISI_TERAKHIR','SLA_ACTUAL','BOOKING_DATE','PLAFOND_DE','PLAFOND_ANALIST','PLAFOND_KOMITE','NAMA_PENJUAL','NAMA_PROJECT',
               'SUMBER_APLIKASI','KATEGORI_JAMINAN','DEVIASI','LEVEL_KOMITE','NAMA_SUB_PROGRAM_RM','MENGAJUKAN_PEMBIAYAAN','NAMA_DEVELOPER',
               'NAMA_AGEN_PROPERTI','NAMA_MARKETING','NO_HP_MARKETING','NAMA_PERUSAHAAN_PEMBERI_REFERAL','CATATAN_RM1','JENIS_PENDAPATAN','KATEGORY',
               'TUJUAN_PEMBIAYAAN','AKAD_PEMBIAYAAN','JENIS_ASSET','NAMA_PERUSAHAAN_NASABAH','NAMA_SUB_PROGRAM_DE','PROGRAM_DE','SUB_PROGRAM_POLICY','SUB_PROGRAM_NON_POLICY']
tracking_fos = laporan_tracking_fos[['APPLICATIONID','PRODUK']]

#FOS inquiry transaction
curs = conn.cursor()
curs.execute("SELECT * FROM xxxRPT.xxx_INQUIRY_TRANSACTION;")
result = curs.fetchall()
fos_inquiry_transaction = pd.DataFrame(result)

fos_inquiry_transaction.columns =['SEQUENCE','NO_APLIKASI','ALTERNATE','TAHAPAN','NASABAH','CABANG','REGION','OLEH','MULAI_TANGGAL','MULAI_WAKTU',
                                  'SELESAI_TANGGAL','SELESAI_WAKTU','AGING','REASON','KETERANGAN','CREATE_TIME']
inquiry_tran = fos_inquiry_transaction

In [ ]:
#Seting working days on hollidays
weekno = datetime.today().weekday()
if weekno > 4: #sabtu-minggu
    print('Weekend!!')
elif weekno > 0 & weekno < 5: #selasa-jumat
    datestring = datetime.strftime(datetime.now()- timedelta(1), '%Y%m%d')
    dtstrng = datetime.strftime(datetime.now()- timedelta(1), '%B, %d %Y')
else:  #senin
    datestring = datetime.strftime(datetime.now()- timedelta(3), '%Y%m%d')
    dtstrng = datetime.strftime(datetime.now()- timedelta(3), '%B, %d %Y')

In [ ]:
#Filter the data

#ambil product
product_appid = tracking_fos[['APPLICATIONID','PRODUK']]
product_appid = product_appid.drop_duplicates(['APPLICATIONID'],keep= 'last')

#Ambil Sequence terbesar
fos = inquiry_tran.sort_values(['SEQUENCE','NO_APLIKASI'], ascending=[False, True])
fos_new = fos.drop_duplicates(['NO_APLIKASI'], keep="first")

#Hapus is kolom ['TAHAPAN'] dan ['SELESAI_TANGGAL'] yang tidah di butuhkan dan 
fos_new = fos_new.loc[~fos_new['TAHAPAN'].isin(["Booked","Booked Manual","Cancel","Canceled",
                                                                "Checking Confirmed","Checking Done","Komite Risk Cancel",
                                                                "Komite Risk Reject","Reject","Send To Debtector",
                                                                "Send To Debtector Failed","SLIK Not Found",
                                                                "Persetujuan Kemampuan Bayar","Komite HC",""])]

fos_new = fos_new[fos_new['SELESAI_TANGGAL'].isnull()]
#tambah kolom THP
THP = {'Legal':'Legal', 'Admin Dropping':'Admin Dropping','Analisa Kredit':'Analisa Kredit','FA Manager':'FA Manager',
          'Komite Risk':'Komite Risk','Komite Risk Start':'Komite Risk Start','OL Preparation':'OL Preparation','Appraisal Entry':'Appraisal',
          'Appraisal Entry - 000xxxxxxxx10101':'Appraisal','Verification Done':'Verification','Verification Review':'Verification','Verified by Officer':'Verification',
          'Verifikasi Review (HC P2R)':'Verification','Waiting for Verification':'Verification','Entri Data':'Entri Data',
          'Prescreen':'Prescreen','BM Screening':'BM Screening','RM 1':'RM 1','RM 2':'RM 2','Settlement':'Settlement','Settlement SPV':'Settlement',
          'Order Phone Verification':'Verification','Checking Done':'Checking Done'}
fos_new["tahapan2"] = fos_new["TAHAPAN"].map(THP)
#tambah kolom produk
fos_new = fos_new.merge(product_appid, how = 'left',
                left_on = 'NO_APLIKASI', right_on = 'APPLICATIONID').drop(columns = ['APPLICATIONID'])

#Isi kolom ['SELESAI_TANGGAL'] dan ['SELESAI_WAKTU'] yang kosong
fos_new['SELESAI_TANGGAL'].fillna((datetime.strftime(datetime.now() - timedelta(1), '%d-%b-%Y')), inplace=True)
fos_new['SELESAI_WAKTU'].fillna(('17:00:00'), inplace=True)

#Buat tangal baru berrdasarkan kolom ['MULAI WAKTU'] dan ['MULAI TANGGAL'] 
fos_new['HR'] = (fos_new['MULAI_WAKTU'].str[:2]).astype('int')
fos_new['MULAI_TANGGAL'] = pd.to_datetime(fos_new['MULAI_TANGGAL'])
fos_new['SELESAI_TANGGAL'] = pd.to_datetime(fos_new['SELESAI_TANGGAL'])
fos_new['MULAI_TANGGAL_BARU'] = np.where((fos_new['HR'] > 12 ),fos_new['MULAI_TANGGAL'] + timedelta(1),
                                                                         fos_new['MULAI_TANGGAL'])
fos_new["DayOfWeek"] = fos_new["MULAI_TANGGAL_BARU"].dt.weekday
fos_new["IsWeekend"] = fos_new["DayOfWeek"] >= 5

fos_new['MULAI_TANGGAL_BARU_SABTU'] = np.where((fos_new['DayOfWeek'] == 5 ),fos_new['MULAI_TANGGAL_BARU']+ timedelta(2) ,
                                          fos_new['MULAI_TANGGAL_BARU'])

fos_new.drop(columns = ['MULAI_TANGGAL_BARU','DayOfWeek','IsWeekend'], inplace=True)

dict = {'MULAI_TANGGAL_BARU_SABTU': 'Tanggal Mulai'}
fos_new.rename(columns=dict,  inplace=True)

#tambah kolom tanggal akhir
fos_new['Tanggal Akhir'] = np.where((fos_new['tahapan2'].isin(['Legal'])), fos_new['Tanggal Mulai']+ timedelta(2),
                     np.where((fos_new['tahapan2'].isin(['Admin Dropping'])), fos_new['Tanggal Mulai']+ timedelta(1), 
                     np.where((fos_new['tahapan2'].isin(['Analisa Kredit'])), fos_new['Tanggal Mulai']+ timedelta(1),
                     np.where((fos_new['tahapan2'].isin(['Komite Risk'])), fos_new['Tanggal Mulai']+ timedelta(1),
                     np.where((fos_new['tahapan2'].isin(['OL Preparation'])), fos_new['Tanggal Mulai']+ timedelta(1),
                     np.where((fos_new['tahapan2'].isin(['Verification'])), fos_new['Tanggal Mulai']+ timedelta(1),
                     np.where((fos_new['tahapan2'].isin(['Entri Data'])), fos_new['Tanggal Mulai'],
                     np.where((fos_new['tahapan2'].isin(['Prescreen'])), fos_new['Tanggal Mulai'],
                     np.where((fos_new['tahapan2'].isin(['RM 1'])), fos_new['Tanggal Mulai'],
                     np.where((fos_new['tahapan2'].isin(['RM 2'])), fos_new['Tanggal Mulai'],                              
                     np.where((fos_new['tahapan2'].isin(['Settlement'])), fos_new['Tanggal Mulai'],
                     np.where((fos_new['tahapan2'].isin(['BM Screening'])), fos_new['Tanggal Mulai'],
                     np.where((fos_new['tahapan2'].isin(['FA Manager'])), fos_new['Tanggal Mulai'],                            
                     fos_new['Tanggal Mulai'])))))))))))))

fos_new["DayOfWeek"] = fos_new["Tanggal Akhir"].dt.weekday
fos_new["IsWeekend"] = fos_new["DayOfWeek"] >= 5

fos_new['MULAI_TANGGAL_AKHIR_SABTU'] = np.where((fos_new['DayOfWeek'] == 5 ),fos_new['Tanggal Akhir']+ timedelta(2) ,
                                          fos_new['Tanggal Akhir'])

fos_new.drop(columns = ['Tanggal Akhir','DayOfWeek','IsWeekend'], inplace=True)

dict = {'MULAI_TANGGAL_AKHIR_SABTU': 'Tanggal Akhir'}
fos_new.rename(columns=dict,  inplace=True)

#tanggal hari libur indonesia
holiday_dates = [dt.date(2022, 4, 29), dt.date(2022, 4, 30), dt.date(2022, 5, 1), dt.date(2022, 5, 2),
                 dt.date(2022, 5, 3), dt.date(2022, 5, 4), dt.date(2022, 5, 5), dt.date(2022, 5, 6),
                 dt.date(2022, 5, 7), dt.date(2022, 5, 8), dt.date(2022, 5, 16), dt.date(2022, 5, 28),
                 dt.date(2022, 6, 1),dt.date(2022, 7, 9),dt.date(2022, 7, 30),dt.date(2022, 8, 17),
                 dt.date(2022, 10, 8),dt.date(2022, 12, 25)]

#Hitung hari kerja berdasarkan Kolom ['new_dt'] dan ['SELESAI TANGGAL']
fos_new['SLA'] = fos_new.apply(lambda x: len(pd.bdate_range(x['Tanggal Mulai'],
                                                           x['SELESAI_TANGGAL'],
                                                           holidays=holiday_dates,
                                                           freq='C',
                                                           weekmask = None)), axis = 1)
fos_new['SLA'] = fos_new['SLA'] - 1
fos_new['SLA'] = np.where((fos_new['SLA'] < 0 ),0 ,fos_new['SLA'])

#tambah kolom flag_sla1
fos_new['flag_sla1'] = np.where((fos_new['SLA'] == 0), '0 BD',
                     np.where((fos_new['SLA'] == 1), '1 BD', 
                     np.where((fos_new['SLA'] == 2), '2 BD',
                     np.where((fos_new['SLA'] >= 3) & (fos_new['SLA'] <= 5),   '3-5 BD',
                     np.where((fos_new['SLA'] >= 6) & (fos_new['SLA'] <= 10),  '6-10 BD',
                     np.where((fos_new['SLA'] >= 11) & (fos_new['SLA'] <= 30), '11-30 BD',
                     np.where((fos_new['SLA'] >= 31) & (fos_new['SLA'] <= 60), '11-60 BD',
                     '> 60 BD')))))))

fos_new['Note'] = np.where((fos_new['tahapan2'].isin(['Legal'])) & (fos_new['SLA']<=2), 'Meet TAT',
                     np.where((fos_new['tahapan2'].isin(['Admin Dropping'])) & (fos_new['SLA']<=1), 'Meet TAT', 
                     np.where((fos_new['tahapan2'].isin(['Analisa Kredit'])) & (fos_new['SLA']<=1), 'Meet TAT',
                     np.where((fos_new['tahapan2'].isin(['Komite Risk'])) & (fos_new['SLA']<=1), 'Meet TAT',
                     np.where((fos_new['tahapan2'].isin(['OL Preparation'])) & (fos_new['SLA']<=1), 'Meet TAT',
                     np.where((fos_new['tahapan2'].isin(['Verification'])) & (fos_new['SLA']<=1), 'Meet TAT',
                     np.where((fos_new['tahapan2'].isin(['Entri Data'])) & (fos_new['SLA']<=0), 'Meet TAT',
                     np.where((fos_new['tahapan2'].isin(['Prescreen'])) & (fos_new['SLA']<=0), 'Meet TAT',
                     np.where((fos_new['tahapan2'].isin(['RM 1'])) & (fos_new['SLA']<=0), 'Meet TAT',
                     np.where((fos_new['tahapan2'].isin(['RM 2'])) & (fos_new['SLA']<=0), 'Meet TAT',                              
                     np.where((fos_new['tahapan2'].isin(['Settlement'])) & (fos_new['SLA']<=0), 'Meet TAT',
                     np.where((fos_new['tahapan2'].isin(['BM Screening'])) & (fos_new['SLA']<=0), 'Meet TAT',
                     np.where((fos_new['tahapan2'].isin(['FA Manager'])) & (fos_new['SLA']<=0), 'Meet TAT',                              
                     'Over TAT')))))))))))))
fos_new['plafon'] = 0

#tambah kolom key dan key1
fos_new['key'] = fos_new['REGION']+fos_new['tahapan2']+fos_new['Note']
fos_new['key2'] = fos_new['PRODUK']+fos_new['key']

fos_new['MULAI_TANGGAL'] = fos_new['MULAI_TANGGAL'].dt.strftime('%d-%b-%Y')
fos_new['SELESAI_TANGGAL'] = fos_new['SELESAI_TANGGAL'].dt.strftime('%d-%b-%Y')

fos_new['Tanggal Mulai'] = fos_new['Tanggal Mulai'].dt.strftime('%d-%b-%Y')
fos_new['Tanggal Akhir'] = fos_new['Tanggal Akhir'].dt.strftime('%d-%b-%Y')

fos_new = fos_new[['NO_APLIKASI','ALTERNATE','TAHAPAN','NASABAH','CABANG','REGION','OLEH','MULAI_TANGGAL','MULAI_WAKTU','SELESAI_TANGGAL',
                'SELESAI_WAKTU','AGING','REASON','KETERANGAN','Tanggal Mulai','Tanggal Akhir','SLA','PRODUK','plafon','flag_sla1','Note',
                   'tahapan2','key','key2']]

list_SLA = ["Admin Dropping","Analisa Kredit","BM Screening","Entri Data","FA Manager","Komite Risk",
            "Legal","OL Preparation","Prescreen","RM 1","RM 2","Verification Review",
            "Waiting for Verification"]

In [ ]:
#update data fosnew to dashboard Excell
app = xw.App()
wb = app.books.open('C:/Users/Report/List as Of 2022.xlsx')
sht = wb.sheets['list']

sht.clear_contents()
sht.range("A1").options(index=False).value=fos_new

tabel = wb.sheets['Summary']
tabel.range("A86:D102").clear_contents()
tabel.range("A86").value = fos_pivot
wb.save()
app.kill()

In [ ]:
# start an instance of Excel
excel = client.Dispatch('Excel.Application')
#save pic untuk email
# make sure you use either a raw string OR escape characters for the backslash. 
# You CANNOT use forward slash format with `win32com`.
path =r'C:/Users/List as Of 2022.xlsx'

# open the work book
wb = excel.Workbooks.Open(path)
sheet = wb.Sheets['Summary']

# copy the target range
copyrange = sheet.Range('A69:D83')
copyrange.CopyPicture(Appearance=1, Format=2)
# grab the saved image from the clipboard and save to working directory
img = ImageGrab.grabclipboard()
        # Save the image to disk
img.save('C:/Users/Report/sla_img.png', 'PNG')
# img.save('paste.jpg', 'JPEG')
wb.Save()
wb.Close()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Save the file in Onedrive 

In [ ]:
#save hasil fosnew
with pd.ExcelWriter('C:/Users/List_inquiry.xlsx') as writer:
    fos_new.to_excel(writer, sheet_name='fos',index=False)

filename = 'List as Of 2022.xlsx'
filename_dest = 'List Over ' + datestring + '.xlsx'
SourcePathName = 'C:/Users/Report/'
SourcePathNameDest  = 'C:/Users/OneDrive - PT. Bangbang/LIst Over SLA/'

#save as menjadi nama baru
src= SourcePathName + filename
dst= SourcePathNameDest + filename_dest
shutil.copy(src,dst)

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## Sending automatic email using outlook


In [ ]:
#Sending Email
# Import modules
import smtplib, ssl
## email.mime subclasses
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
### Add new subclass for adding attachments
from email.mime.application import MIMEApplication
## The pandas library is only for generating the current date, which is not necessary for sending emails
import pandas as pd

# Define the HTML document
# Add an image element
##############################################################
html = '''
<html>
<body>
    <h1 style="font-family:verdana;"><b>Report Progress</b></h1>
    <p style="font-family:verdana;"><b><i>Assalaamu'alaikum Warahmatullahi Wabarakatuh</i></b>,<br>Salam, Bapak dan Ibu,<br></p>
    <p style="font-family:verdana;">Berikut disampaikan Report Daily as of '''+dtstrng+'''</p>
    <img src='cid:myimageid' width="300" height="250">
    <p style="font-family:verdana;">Report lengkap nya ada di OD <a href=" https://bangbangcoid-my.sharepoint.com/personal/20200193_bangbang_co_id/_layouts/15/onedrive.aspx?id=%2Fpersonal%2F20200193%5Fbangbang%5Fco%5Fid%2FDocuments%2FLIst%20Over%20SLA&ct=1659425371448&or=OWA%2DNT&cid=cea6f636%2Da889%2D1eb4%2D25d5%2D7e436fd34e22&ga=1/">Klik disini</a> </p>                                    
    <p style="font-family:verdana;">Demikian disampaikan, terima kasih. Semoga sehat selalu.<br>Wassalaamu'alaikum, Wr. Wb<br></p>
    <p style="font-family:verdana;">Terima Kasih,</p>
    <p style="font-family:verdana;">mis.fop<br>MIS Financing Operations<br><i>This Email sent by Task Scheduller</i><br></p>
</body>
</html>
    '''
##############################################################

# Define a function to attach files as MIMEApplication to the email
    ## Add another input extra_headers default to None
##############################################################
def attach_file_to_email(email_message, filename, extra_headers=None):
    # Open the attachment file for reading in binary mode, and make it a MIMEApplication class
    with open(filename, "rb") as f:
        file_attachment = MIMEApplication(f.read())
    # Add header/name to the attachments    
    file_attachment.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )
    # Set up the input extra_headers for img
      ## Default is None: since for regular file attachments, it's not needed
      ## When given a value: the following code will run
         ### Used to set the cid for image
    if extra_headers is not None:
        for name, value in extra_headers.items():
            file_attachment.add_header(name, value)
    # Attach the file to the message
    email_message.attach(file_attachment)
##############################################################    


recipients =["nunxxx@bangbang.co.id", "nadzira.khxxx@bangbang.co.id", "priska.ratnadhiaxxx@bangbang.co.id", "rixxxx@bangbang.co.id", "tommi.pxxxxa@bangbang.co.id", "andris.susxxxx@bangbang.co.id", "dennis.farxxxx@bangbang.co.id", "iqbal.faxxxx@bangbang.co.id", "erika.wixxxx@bangbang.co.id", "arief.rosdiyatxxxx@bangbang.co.id", "mesti.kxxxx@bangbang.co.id", "rdhi.praxxxx@bangbang.co.id", "nasruxxxx@bangbang.co.id", "fauz.atxxxx@bangbang.co.id"]
cc_recipients =["ibnu.uxxx@bangbang.co.id", "sasxxx.putra@bangbang.co.id", "vinny.anxxx@bangbang.co.id", "nika.paraxxxx@bangbang.co.id", "mitha.djayanti@bangbang.co.id" , "sendi.puxxx@bangbang.co.id"]


# recipients = ["xxxxx.putro@bangbang.co.id"]
# cc_recipients =["xxxxx.putro@bangbang.co.id"]

# Set up the email addresses and password. Please replace below with your email address and password
email_from = 'mis.xxx@bangbang.co.id'
password = 'XXxxx@2022'
email_to = ', '.join(recipients) 
cc_to = ', '.join(cc_recipients) 
port = 587  # For starttls
smtp_server = "smtp.office365.com"

# Generate today's date to be included in the email Subject
# date_str = pd.Timestamp.today().strftime('%Y-%m-%d')

# Create a MIMEMultipart class, and set up the From, To, Subject fields
email_message = MIMEMultipart()
email_message['From'] = email_from
email_message['To'] = email_to
email_message['Cc'] = cc_to
email_message['Subject'] = f'List FOS as Of - {dtstrng}'

# Attach the html doc defined earlier, as a MIMEText html content type to the MIME message
email_message.attach(MIMEText(html, "html"))

# Attach more (documents)
  ## Apply function with extra_header on chart.png. This will render chart.png in the html content
##############################################################
attach_file_to_email(email_message, 'C:/sla_img.png', {'Content-ID': '<myimageid>'})
##############################################################
# attach_file_to_email(email_message, 'List Over SLA ' + datestring + '.xlsx')
# attach_file_to_email(email_message, 'List_Akta_Jamin.xlsx')

# Convert it as a string
email_string = email_message.as_string()
xx = datetime.strftime(datetime.now()-timedelta(), format='%Y-%m-%d %H:%M:%S')

# Connect to the Gmail SMTP server and Send Email
context = ssl.create_default_context()
with smtplib.SMTP(smtp_server, port) as server:
    server.ehlo()  # Can be omitted
    server.starttls(context=context)
    server.ehlo()  # Can be omitted
    server.login(email_from, password)
    # server.sendmail(email_from, email_to, email_string)
    for recipient in recipients:
        server.sendmail(email_from, recipient, email_string)
    for cc_recipient in cc_recipients:
        server.sendmail(email_from, cc_recipient, email_string)
    print('Report SLA ==> Email sent! Please Check Inbox')
    print(xx)
    server.close()    
